# Car Price Prediction::

Download dataset from this link:

https://www.kaggle.com/hellbuoy/car-price-prediction

# Problem Statement::

A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts.

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. Specifically, they want to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. The company wants to know:

Which variables are significant in predicting the price of a car
How well those variables describe the price of a car
Based on various market surveys, the consulting firm has gathered a large data set of different types of cars across the America market.

# task::
We are required to model the price of cars with the available independent variables. It will be used by the management to understand how exactly the prices vary with the independent variables. They can accordingly manipulate the design of the cars, the business strategy etc. to meet certain price levels. Further, the model will be a good way for management to understand the pricing dynamics of a new market.

# WORKFLOW ::

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

4.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

5.Compilation Step (Note : Its a Regression problem , select loss , metrics according to it)
6.Train the Model with Epochs (100) and validate it

7.If the model gets overfit tune your model by changing the units , No. of layers , activation function , epochs , add dropout layer or add Regularizer according to the need .

8.Evaluation Step

9.Prediction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers,utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data=pd.read_csv("data/Car Price Prediction/CarPrice_Assignment.csv")

In [3]:
data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [5]:
x=data.drop(["price","car_ID","CarName"],axis=1).copy()
y=data["price"].copy()

In [6]:
{column: list(x[column].unique()) for column in x.columns if x.dtypes[column]=='object'}

{'fueltype': ['gas', 'diesel'],
 'aspiration': ['std', 'turbo'],
 'doornumber': ['two', 'four'],
 'carbody': ['convertible', 'hatchback', 'sedan', 'wagon', 'hardtop'],
 'drivewheel': ['rwd', 'fwd', '4wd'],
 'enginelocation': ['front', 'rear'],
 'enginetype': ['dohc', 'ohcv', 'ohc', 'l', 'rotor', 'ohcf', 'dohcv'],
 'cylindernumber': ['four', 'six', 'five', 'three', 'twelve', 'two', 'eight'],
 'fuelsystem': ['mpfi', '2bbl', 'mfi', '1bbl', 'spfi', '4bbl', 'idi', 'spdi']}

In [7]:
x["doornumber"]=x["doornumber"].apply(lambda x: 0 if x == 'two' else 1)

In [8]:
numeric_ordering ={'two':2,'three':3,'four':4,'five':5,'six':6,'eight':8,'twelve':12}

In [9]:
x["cylindernumber"] =x["cylindernumber"].replace(numeric_ordering)

In [10]:
def binary_encode(x, columns, positive_values):
    x =x.copy()
    for column, positive_value in zip(columns, positive_values):
        x[column] = x[column].apply(lambda x: 1 if x == positive_value else 0)
    return x

In [11]:
binary_features=['fueltype','aspiration','enginelocation',]
binary_positive_values=['diesel','turbo','front']
x=binary_encode(x,columns=binary_features,positive_values=binary_positive_values)

In [12]:
binary_features = ['fueltype','aspiration','enginelocation']
binary_positive_values=['diesel','turbo','front']
x=binary_encode(x,columns=binary_features,positive_values=binary_positive_values)

In [13]:
def onehot_encode(x, columns, prefixes):
    x = x.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(x[column], prefix=prefix)
        x=pd.concat([x, dummies], axis=1)
        x=x.drop(column, axis=1)
    return x

In [14]:
nominal_features=['carbody','drivewheel','enginetype','fuelsystem']
prefixes=['MK','BS','DW','ET','FS']
x=onehot_encode(x,columns=nominal_features,prefixes=prefixes)

In [15]:
x

,symboling,fueltype,aspiration,doornumber,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,...,DW_ohcv,DW_rotor,ET_1bbl,ET_2bbl,ET_4bbl,ET_idi,ET_mfi,ET_mpfi,ET_spdi,ET_spfi
0,3,0,0,0,0,88.6,168.8,64.1,48.8,2548,...,0,0,0,0,0,0,0,1,0,0
1,3,0,0,0,0,88.6,168.8,64.1,48.8,2548,...,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,94.5,171.2,65.5,52.4,2823,...,1,0,0,0,0,0,0,1,0,0
3,2,0,0,1,0,99.8,176.6,66.2,54.3,2337,...,0,0,0,0,0,0,0,1,0,0
4,2,0,0,1,0,99.4,176.6,66.4,54.3,2824,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,0,0,1,0,109.1,188.8,68.9,55.5,2952,...,0,0,0,0,0,0,0,1,0,0
201,-1,0,0,1,0,109.1,188.8,68.8,55.5,3049,...,0,0,0,0,0,0,0,1,0,0
202,-1,0,0,1,0,109.1,188.8,68.9,55.5,3012,...,1,0,0,0,0,0,0,1,0,0
203,-1,0,0,1,0,109.1,188.8,68.9,55.5,3217,...,0,0,0,0,0,1,0,0,0,0


In [16]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state=123)

In [17]:
scaler = StandardScaler()
scaler.fit(x_train)

StandardScaler()

In [18]:
x_train =pd.DataFrame(scaler.transform(x_train),columns=x_train.columns)
x_test=pd.DataFrame(scaler.transform(x_test),columns=x_test.columns)

In [19]:
part_x_train,x_val,part_y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=2)

In [20]:
model = Sequential()
model.add(Dense(12, input_dim=42, kernel_initializer='normal', activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

In [21]:
history=model.fit(part_x_train,part_y_train,epochs=100,validation_data=(x_val,y_val),batch_size=512)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 263501760.0000 - mse: 263501760.0000 - mae: 14127.9902 - val_loss: 334181920.0000 - val_mse: 334181920.0000 - val_mae: 14966.4561
Epoch 2/100
1/1 [==============================] - 0s 76ms/step - loss: 263500992.0000 - mse: 263500992.0000 - mae: 14127.9697 - val_loss: 334181024.0000 - val_mse: 334181024.0000 - val_mae: 14966.4355
Epoch 3/100
1/1 [==============================] - 0s 67ms/step - loss: 263500208.0000 - mse: 263500208.0000 - mae: 14127.9463 - val_loss: 334180064.0000 - val_mse: 334180064.0000 - val_mae: 14966.4102
Epoch 4/100
1/1 [==============================] - 0s 87ms/step - loss: 263499440.0000 - mse: 263499440.0000 - mae: 14127.9229 - val_loss: 334179104.0000 - val_mse: 334179104.0000 - val_mae: 14966.3877
Epoch 5/100
1/1 [==============================] - 0s 91ms/step - loss: 263498608.0000 - mse: 263498608.0000 - mae: 14127.9014 - val_loss: 334178080.0000 - val_mse: 334178080.0000 - val_mae: 149

1/1 [==============================] - 0s 67ms/step - loss: 263458272.0000 - mse: 263458272.0000 - mae: 14126.7646 - val_loss: 334126304.0000 - val_mse: 334126304.0000 - val_mae: 14965.1338
Epoch 42/100
1/1 [==============================] - 0s 70ms/step - loss: 263456768.0000 - mse: 263456768.0000 - mae: 14126.7227 - val_loss: 334124352.0000 - val_mse: 334124352.0000 - val_mae: 14965.0879
Epoch 43/100
1/1 [==============================] - 0s 58ms/step - loss: 263455216.0000 - mse: 263455216.0000 - mae: 14126.6787 - val_loss: 334122368.0000 - val_mse: 334122368.0000 - val_mae: 14965.0391
Epoch 44/100
1/1 [==============================] - 0s 68ms/step - loss: 263453680.0000 - mse: 263453680.0000 - mae: 14126.6348 - val_loss: 334120320.0000 - val_mse: 334120320.0000 - val_mae: 14964.9893
Epoch 45/100
1/1 [==============================] - 0s 76ms/step - loss: 263452064.0000 - mse: 263452064.0000 - mae: 14126.5889 - val_loss: 334118272.0000 - val_mse: 334118272.0000 - val_mae: 14964.941

1/1 [==============================] - 0s 84ms/step - loss: 263372480.0000 - mse: 263372480.0000 - mae: 14124.4092 - val_loss: 334014880.0000 - val_mse: 334014880.0000 - val_mae: 14962.5420
Epoch 82/100
1/1 [==============================] - 0s 76ms/step - loss: 263369520.0000 - mse: 263369520.0000 - mae: 14124.3291 - val_loss: 334011072.0000 - val_mse: 334011072.0000 - val_mae: 14962.4551
Epoch 83/100
1/1 [==============================] - 0s 67ms/step - loss: 263366496.0000 - mse: 263366496.0000 - mae: 14124.2480 - val_loss: 334007200.0000 - val_mse: 334007200.0000 - val_mae: 14962.3662
Epoch 84/100
1/1 [==============================] - 0s 66ms/step - loss: 263363424.0000 - mse: 263363424.0000 - mae: 14124.1670 - val_loss: 334003168.0000 - val_mse: 334003168.0000 - val_mae: 14962.2744
Epoch 85/100
1/1 [==============================] - 0s 66ms/step - loss: 263360336.0000 - mse: 263360336.0000 - mae: 14124.0820 - val_loss: 333999168.0000 - val_mse: 333999168.0000 - val_mae: 14962.184

In [22]:
model.evaluate(x_test,y_test)

2/2 [==============================] - 0s 0s/step - loss: 151872688.0000 - mse: 151872688.0000 - mae: 10916.0879


[151872688.0, 151872688.0, 10916.087890625]

In [23]:
model.predict(x_test).sum()

283.55194